In [1]:
print("OK!")

OK!


In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [5]:
PINECONE_API_KEY="6f043c11-9675-47ee-b7ab-24161ed91eee"
PINECONE_API_ENV="aws-us-east-1"

In [6]:
#Extract data from the pdf file
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents


In [ ]:
load_pdf("data/")